<a href="https://colab.research.google.com/github/saparbayev-azizbek-12/bi-and-ai-talents-dl/blob/main/lesson-10/lesson_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate loss on 4-gram

In [1]:
%%capture
!wget https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt

In [ ]:
names = open('names.txt').read().splitlines()

In [ ]:
import torch

In [ ]:
vocab = sorted(set(''.join(names) + '.'))

In [ ]:
stoi = {chr:idx for idx, chr in enumerate(vocab)}
itos = {idx:chr for chr, idx in stoi.items()}

In [ ]:
N4 = torch.zeros((27, 27, 27, 27), dtype=torch.long)

for name in names:
  name = ["."] * 3 + list(name) + ["."]
  for chr1, chr2, chr3, chr4 in zip(name, name[1:], name[2:], name[3:]):
    idx1 = stoi[chr1]
    idx2 = stoi[chr2]
    idx3 = stoi[chr3]
    idx4 = stoi[chr4]
    N4[idx1, idx2, idx3, idx4] += 1

In [ ]:
P4 = (N4 + 1).float() / N4.sum(dim=3, keepdim=True)

In [ ]:
P4

tensor([[[[3.1218e-05, 1.3770e-01, 4.0802e-02,  ..., 4.2144e-03,
           1.6733e-02, 2.9033e-02],
          [2.2676e-04, 4.7166e-02, 4.3311e-02,  ..., 6.3492e-03,
           3.9456e-02, 3.4694e-02],
          [7.6570e-04, 1.3017e-01, 7.6570e-04,  ..., 7.6570e-04,
           3.8285e-03, 7.6570e-04],
          ...,
          [7.4627e-03, 4.3284e-01, 7.4627e-03,  ..., 1.4925e-02,
           1.3433e-01, 8.9552e-02],
          [1.8692e-03, 4.6168e-01, 1.8692e-03,  ..., 1.8692e-03,
           1.8692e-03, 5.6075e-03],
          [1.0764e-03, 4.9193e-01, 1.0764e-03,  ..., 1.0764e-03,
           9.9031e-02, 2.1529e-03]],

         [[       inf,        inf,        inf,  ...,        inf,
                  inf,        inf],
          [4.8309e-03, 4.8309e-03, 2.8986e-02,  ..., 4.8309e-03,
           6.2802e-02, 9.6618e-03],
          [5.2632e-03, 3.6842e-02, 7.8947e-02,  ..., 5.2632e-03,
           2.6316e-02, 5.2632e-03],
          ...,
          [3.7037e-02, 1.1111e-01, 3.7037e-02,  ..., 1.1111

In [ ]:
log_likelihood = 0
n = 0

for name in names:
  name = ["."] * 3 + list(name) + ["."]
  for chr1, chr2, chr3, chr4 in zip(name, name[1:], name[2:], name[3:]):
    prob = P4[stoi[chr1], stoi[chr2], stoi[chr3], stoi[chr4]]
    log_likelihood += torch.log(prob)
    n += 1

nll = -log_likelihood / n
nll

tensor(1.7951)

# Neural Networks Approach

- Move model and X, Y into GPU (done)
- Make inferene using trained model
- Try the model with more complex settings
- Try to beat the statistical bigram model (test set loss = 2.5936)

In [ ]:
import torch.nn.functional as F

In [ ]:
torch.cuda.is_available()

True

In [ ]:
split_per = 80
split_num = len(names) * split_per // 100
tr_names = names[:split_num]
ts_names = names[split_num:]

In [ ]:
X = []
Y = []

for name in tr_names:
  name = ['.'] + list(name) + ['.']
  for ch1, ch2 in zip(name, name[1:]):
    X.append(stoi[ch1])
    Y.append(stoi[ch2])

X = torch.tensor(X, dtype=torch.long, device='cuda')
Y = torch.tensor(Y, dtype=torch.long, device='cuda')

In [ ]:
X_enc = F.one_hot(X, num_classes=27).float()


In [ ]:
W1 = torch.randn(64, 27, requires_grad=True, device='cuda')
b1 = torch.randn(27, requires_grad=True, device='cuda')

input_layer = torch.nn.Linear(27, 64, device='cuda')

for i in range(1000):
  hidden_layer = input_layer(X_enc)
  logits = hidden_layer @ W1 + b1

  nll = F.cross_entropy(logits, Y)

  if i % 100 == 0:
      print("Loss:", nll.item())

  W1.grad = None
  input_layer.zero_grad()
  nll.backward()
  W1.data += -1 * W1.grad
  input_layer.weight.data += -1 * input_layer.weight.grad
  input_layer.bias.data += -1 * input_layer.bias.grad

Loss: 4.162471771240234
Loss: 2.6078336238861084
Loss: 2.465646266937256
Loss: 2.445481777191162
Loss: 2.436854839324951
Loss: 2.430133819580078
Loss: 2.429163694381714
Loss: 2.4274251461029053
Loss: 2.426727771759033
Loss: 2.426867961883545
